# Introduction

The purpose of my project is to investigate different machine learning methods by using them to predict car prices. In this project I will be using a data set relating to different features of a car (e.g. engine size or amount of doors) and using this data to train certain machine learning models and evaluate them. I am doing this to test out machine learning models because price is a practical and easy numerical amount to understand so it will be easy for me to experiment with different models and see where else machine learning can be implementented. 

For this project I am going to be comparing only two different machine learning models: the random forest and the decision tree regressor. By the end of this project, I will be able to assess how good these two models are by evaluating the accuracy of their predictions

Also, by doing this project I will be investigating how different features of a car affect it's price

### Hypotheses:

* I predict that the random forest will be the most accurate machine learning model because of past experience and research although I predict that the difference between the two models won't be too big.

* I predict that the features 'engine size' and 'horsepower' will have the strongest affect on the price predictions because most of the cars have similar features like 'car width', 'car height' and 'car length'.

* I predict that my model will be accurate in making prediction because I think that a car features greatly affect it's price rather then just what brand it is. Due to the average car price in my dataset being $13276, i think an accurate prediction for this dataset would be around 500 - 1000 off the target.

During this project will go through my use of the 7 steps of machine learning: Gathering data, Preparing data, Choosing a model, Training, Evaluation and Parameter tuning and prediction to evaluate random forests and decision tree regressors

# Setting Up

Before I began this project, had to import some libraries which are shown in the code below

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt # data visualisation
from sklearn.metrics import mean_absolute_error # mean absolute error for evaluating the models
from sklearn.model_selection import train_test_split # for spliting data into training and validation 


# Input data files are available in the read-only "../input/" directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Shows all files under the input directory


# Gathering Data

Now I added my data to the project and read it using Panda DataFrame. I then displayed my data using the describe function

In [ ]:
predict_car_price = '../input/car-price-prediction/CarPrice_Assignment.csv'

# Created a new Pandas DataFrame with our data
predict_car_price_data = pd.read_csv(predict_car_price)

predict_car_price_data.describe(include='all')

The table above displays some information about the data I have chosen. I chose this data because I could predict something numerical (price) and because it had a large amount of features I could chose from. This data set is suitable for making predictions with a decision tree regressor or a random forest as most of it's features are numerical and the ones that are categorical only have a few different values (e.g. the features fueltype, aspiration, doornumber, carbody, drivewheel and enginelocation only have two different values) meaning that they can easily be turned into binary form using One Hot Encoding. 

Even though there were a lot of numerical features I could have chosen to predict, I chose to predict car price for this dataset because I think this is the featue that is the most affected by the other features. For example, I could have chosen to predict the cars' horse power as the data was numerical and had a large range although I didn't think some of the features like car height and wheel base would have affected this feature. 

Most of the numerical features in this data set relate to either the car's size or the car's mechanics while the categorical features consist of a range of things relating to the cars

### Observations:

* Each coumn has a count of 205 meaning there are 205 cars in the data set

* The dataset consists of 147 different car types/brands 

* The minimum car price is 5,118

* The max car price is is 454,000

* The most common car type in the dataset is the Peugeot 504

# Preparing the Data

Before I started training my model I had to prepare my data. Since my data has no missing values or outliers, I dont have to add any code to fix these things before I begin working with the data.

### Selecting the Columns:

First, I made a list of all the features I would like to use for making my predicitions. All of the features I chose are numerical as it is easier to use numerical data in a decision tree although I added one categorical feature ('fueltype') because I thought this was important and it works because it only has two values.

In [ ]:
selected_columns = ['wheelbase', 'fueltype', 'carlength', 'carwidth', 'carheight', 'curbweight', 'enginesize', 'boreratio', 'stroke', 'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg', 'price']

# Create our new training set containing only the features we want
prepared_data = predict_car_price_data[selected_columns]

# Displayed the data again with only chosen features
prepared_data.describe(include='all')

### Seperating Data:

Next I seperated the target from the rest of the features i'm going to use for making predictions.

In [ ]:
# Separated out the prediction target 
y = prepared_data.price

# Dropped the target column from the original dataframe
X = prepared_data.drop('price', axis=1)

I then seperated the data again into the training and validation data. This is needed in machine learning because to be able to evaluate your model after training it you need some data the model hasn't seen before. The 'train test split' is done by the function I imported and for this dataset the split is 153 training data to 52 validation data. This split is done this way because you need to make sure that you have sufficent data to train your model so you aren't underfitting it but you also need to make sure that is enough validation data so that you can accurately evaluate your data and so that you aren't over fitting the model.

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

val_X.describe()

### One Hot Encode Categorical Data:

The last step in preparing my data was changing the categorical 'fueltype' feature into binary form. For this I used One Hot Encode. 

Binary form means data using 0's and 1's indicating 'True' or 'False'. It's used for machine learning becuase some machine learning models (like the ones we are using) can't take categorical data. For example:

<img src="https://i.imgur.com/mtimFxh.png" alt="One Hot Encoding" style="width:600px">

In my project, two new columns were made replacing the fuel type coloum: 'fueltype_diesel' and 'fueltype_gas'. 

In [ ]:
one_hot_train_X = pd.get_dummies(train_X)
one_hot_val_X = pd.get_dummies(val_X)

one_hot_train_X.head()

# Choosing the Models

Like I mentioned before, the two models I will be using in the project are a decision tree regressor and a random forest. The code below imports these two models 

In [ ]:
from sklearn.tree import DecisionTreeRegressor ,plot_tree # plot tree is for the displaying of our decison tree model
from sklearn.ensemble import RandomForestRegressor

# Training the Models

The next step in completing my models is training them. For this I will use the 'train_X' and the 'train_y' data sets.

### Decision Tree Regressor:

A decision tree is a machine learning model that uses a flow cahrt-like tree structure to split data into categories for predictions and a decision tree regressor is a type of decision tree that makes prediction in numerical (continueous) form. There are three main parts of a decision tree: The root node (the first/top node on the decision tree), the branch node (the other nodes on the tree that split the data) and the leaf nodes (the nodes that predict an outcome). The tree works by spliting the dataset at each node by using a feature (e.g one node may say horsepower =< 1000) until the max depth is reached. The program I have added to my project does most of the work in making the decision tree as it using its own equations and formulas to find out which feature should go on each node and which values the data should be split at. 

Decision trees are useful when there is no evident or suspected specific relationship between the features and the outcome (e.g. linear or exponential relationship) or when there are many features in a dataset. A decision tree classifier is suitable for my dataset as there are many features and I am unsure of the relationships in the data.

The code trains my decisiontree model and displays it.

In [ ]:
car_model = DecisionTreeRegressor(random_state=1, max_depth=3)
car_model.fit(one_hot_train_X, train_y) # trains the tree using the training data

# Displays the model 

plt.figure(figsize = (20,10)) 
plot_tree(car_model,
          feature_names=train_X.columns,
          filled=True)
plt.show()

### Observations:

* The decision tree with a max depth of 3 had 7 leaf nodes meaning there were 7 different prediction groups. This tells us that the model is very underfit because there are only 7 different groups for a dataset with 205 values
 
* I can see that the decision tree program uses MSE (Mean Squared Error) and that the highest MSE is 10020603 (or 3165 when not squared). This is not a good result considering that our average car price is 13276
 
* I can see that not every feature was used in making this deciosn tree which is not good as it lowers the accuracy of decisions. Not every feature was used because the tree had a small max depth and didnt have enough nodes for every feature

### Random Forest:

A random forest is similar to a decision tree as the model makes its prediction by using many decision trees and averaging the predictions of each component tree. The features on each of these tree are put at random node locations 

Due to random forests just being a large amount of decision tree, it takes the same data as a decision tree regressor meaning it is also suitable with our dataset. 

The code below trains a random forest model. Due to random forests being made of many different decision trees, it can be display in this notebook.

In [ ]:
forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(one_hot_train_X, train_y)

# Evaluation



Now that I have trained both models I can move on to the next step which is evaluating the models. To do this, we let the model make predictions using 'val_X' and see if these predictions are correct by comparing them to 'val_y'. To measure each models accuracy, we use it's MAE (Mean Absolute Error) when using the validation data. To formulate MAE, the function 'mean_absolute_error' calculates how much each value is off (in my case how far off each prediction is to the val_y value), adds all the values up in absolute form (meaning a negative numbers turn into a positive number) and divides the results by the number of rows to find the average. 

The code below shows the MAE for both the decision tree regressor and the random forest

In [ ]:
# Make validation predictions and calculate mean absolute error
val_predictions = car_model.predict(one_hot_val_X)
val_mae = mean_absolute_error(val_predictions, val_y)
print("Validation MAE when not specifying max_leaf_nodes for the decision tree regressor: {:.0f}".format(val_mae))

forest_preds = forest_model.predict(one_hot_val_X)
forest_mae = mean_absolute_error(val_y, forest_preds)
print("Validation MAE when not specifying max_leaf_nodes for the random forest: {:.0f}".format(forest_mae))

This output above tells me that the random forest model is more accurate then the decision tree regressor as it has a lower MAE. This is the case because random forests consist of many trees while a decision tree regressor is just one meaning the random forest is a more detailed model.

# Hyperparameter Tuning

In machine learning, hyperparameter tuning is the tuning of a models hyperparameters to reach the model's optimal performance. A hyperparameter is a parameter whose value is used to control a models learning process. 

For our machine learning model there aren't many hyperparameters we can change due to the simplicity of our model. Two hyperparameters we can change for both models are the features and the tree depth although I won't change any of the features as I have already gone through preparing the data and training.

### Decision Tree Regressor:

The code and output below shows me experimenting with multiple different tree depths for the decison tree classifier and comparing their MAE

In [ ]:
def get_mae(max_leaf_nodes, one_hot_train_X, one_hot_val_X, train_y, val_y):
    test_model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    test_model.fit(one_hot_train_X, train_y)
    preds_val = test_model.predict(one_hot_val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

# compare MAE with differing values of max_leaf_nodes
for max_leaf_nodes in [3,5,7,10,15,20,25,30,35,40,50,60,70,80,100,120,140,160,200]:
    my_mae = get_mae(max_leaf_nodes, one_hot_train_X, one_hot_val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

Although the results are farely inconsistent, I can see that for the max depths I tested the higher the depth is the lower the MAE is. The results are inconsist because our model there isn't a consistent relationship between our feature and our target like a linear or an exponential graph would have. Although we know that the higher the depth is the less the MAE is, I need to fide the point were the model has become over fit so that I can find the optimal tree depth 

In [ ]:
for max_leaf_nodes in [200, 240, 280, 320, 360, 400, 450, 500, 600, 700, 800, 1000, 1500, 2000, 4000, 10000]:
    my_mae = get_mae(max_leaf_nodes, one_hot_train_X, one_hot_val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

This output shows me that the decision tree program stops adding leaf nodes once it has reached its optimal amount even if the max depth is increased. I will use the smallest max depth that has the MAE of 1615 (160) as optimal tree depth.

### Random Forest:

Next I will do the same process with my random forest

In [ ]:
def get_mae2(max_leaf_nodes, one_hot_train_X, one_hot_val_X, train_y, val_y):
    forest_model = RandomForestRegressor(random_state=1)
    forest_model.fit(one_hot_train_X, train_y)
    preds_val2 = forest_model.predict(one_hot_val_X)
    mae2 = mean_absolute_error(val_y, preds_val2)
    return(mae2)

# compare MAE with differing values of max_leaf_nodes
for max_leaf_nodes in [1,2,3,4,5,100]:
    my_mae2 = get_mae2(max_leaf_nodes, one_hot_train_X, one_hot_val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae2))

This shows output shows me that the accuracy of a random forest does not depend of the amount of leaf nodes. This may be because when the model makes prediction by getting the average result from many different decision trees, the average for trees with 1 depth and trees with 100 depth is the same due to the large number of trees made in a random forest. 

# Conclusion

Through the 8 steps of machine learning, I was able to achieve the purpose of my investigation which was to investigate different machine learning models (specifically the decision tree regressor and the random) by using them to predict car price. 

The results show me that my first hypothesis about which model would be the most accurate was correct as my random forest had the lowest MAE. I couldn't find out the answer to my second hypothesis as I can't measure which feature had the biggest affect when making predictions although I noticed that 'horsepower' and 'enginsize' weren't in the decision tree with a max depth of 3 meaning that they may not have been the most important features. My last hypothesis was wrong has our lowest MAE was 1362 while i predicted that the MAE would be between 500 and 1000.

Although the predictions weren't as accurate as I thought they would be, I still believe the random forest model was a good model for predicting car price with this data set. Things I could have done to improve my prediction could have been adding more of the categorical features or researching about other hyper parameters I could have tuned. 

In conclusion, I learnt that the features of a car have a significant affect on a car price and that the machine learning models, random forest and decision tree regressor, are suitable for making prediction with my dataset.